In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [5]:
test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [161]:
X_train = train_df['title'].values
X_test = test_df['title'].values
Y_train = train_df['target'].astype(int).values

In [162]:
np.sum(Y_train == 0)

118594

In [163]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [164]:
vectorizer = CountVectorizer(
    analyzer='word',
    strip_accents='unicode',
)
model = MultinomialNB()

In [165]:
X_train_vectorized = vectorizer.fit_transform(X_train)

>>> vectorizer = CountVectorizer()
>>> X = vectorizer.fit_transform(corpus)
>>> vectorizer.get_feature_names_out()
array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], ...)
>>> print(X.toarray())
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
>>> vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
>>> X2 = vectorizer2.fit_transform(corpus)
>>> vectorizer2.get_feature_names_out()
array(['and this', 'document is', 'first document', 'is the', 'is this',
       'second document', 'the first', 'the second', 'the third', 'third one',
       'this document', 'this is', 'this the'], ...)
 >>> print(X2.toarray())
 [[0 0 1 1 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 0 0 1]]

In [166]:
feature_names = np.array(vectorizer.get_feature_names_out())

In [167]:
id_ = 42
x_vector = X_train_vectorized.getrow(id_).toarray()

In [168]:
np.unique(x_vector, return_counts=True)

(array([0, 1]), array([188766,      8]))

In [169]:
# pd.DataFrame.value_counts(pd.DataFrame.from_dict(x_vector))

In [170]:
%%time
model.fit(X_train_vectorized, Y_train)
Y_pred = model.predict(X_train_vectorized)

CPU times: user 36.3 ms, sys: 5.56 ms, total: 41.8 ms
Wall time: 40.3 ms


In [171]:
from sklearn.metrics import f1_score

In [172]:
f1_score(Y_train, Y_pred)

0.9039125916355363

In [173]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe


In [123]:
from sklearn.neighbors import KNeighborsClassifier

In [198]:
neigh = KNeighborsClassifier(n_neighbors=1)

In [199]:
neigh.fit(X_train_vectorized, Y_train)

KNeighborsClassifier(n_neighbors=1)

In [200]:
prediction_kNN = neigh.predict(X_train_vectorized)

In [201]:
f1_score(Y_train, prediction_kNN)

0.9920639656304077

In [202]:
test_df["target"] = neigh.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("KNeighborsClassifier.csv", index=False)

In [193]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_vectorized, Y_train)

SVC(kernel='linear')

In [194]:
%%time
prediction_SVC = svclassifier.predict(X_train_vectorized)

CPU times: user 1min 48s, sys: 680 ms, total: 1min 48s
Wall time: 1min 49s


In [196]:
f1_score(Y_train, prediction_SVC)

0.9921350007533524

In [195]:
test_df["target"] = svclassifier.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("SVC.csv", index=False)

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
from numpy import mean
from numpy import std
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
# define the model
LGBmodel = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(LGBmodel, X_train_vectorized, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))